In [1]:
def INDEX_ASSEMBLY(TYPE_ELEMENT):
    if TYPE_ELEMENT == "BAR_2D_1DOF":  
        TOTAL_DOF_NODE = 1
        TOTAL_NODES_ELEMENT = 2
        ARRAY_DOF_ACTIVE = np.array([[1, 0, 0]])
    elif TYPE_ELEMENT == "TRUSS_2D_4DOF":   
        TOTAL_DOF_NODE = 2
        TOTAL_NODES_ELEMENT = 2
        ARRAY_DOF_ACTIVE = np.array([[1, 1, 0]])
    elif TYPE_ELEMENT == "BEAM_2D_4DOF":   
        TOTAL_DOF_NODE = 2
        TOTAL_NODES_ELEMENT = 2
        ARRAY_DOF_ACTIVE = np.array([[0, 1, 1]])
    elif TYPE_ELEMENT == "FRAME_2D_6DOF": 
        TOTAL_DOF_NODE = 3
        TOTAL_NODES_ELEMENT = 2
        ARRAY_DOF_ACTIVE = np.array([[1, 1, 1]])
    COLUMN_MATERIAL_INDEX = TOTAL_NODES_ELEMENT + 0
    COLUMN_SECTION_INDEX = TOTAL_NODES_ELEMENT + 1
    TOTAL_DOF_ELEMENT = TOTAL_DOF_NODE * TOTAL_NODES_ELEMENT
    return TOTAL_DOF_NODE, TOTAL_NODES_ELEMENT, TOTAL_DOF_ELEMENT, COLUMN_MATERIAL_INDEX, COLUMN_SECTION_INDEX, ARRAY_DOF_ACTIVE

def GLOBAL_DOF_ASSEMBLY(TYPE_ELEMENT, TOTAL_DOF_NODE, TOTAL_NODES, TOTAL_ELEMENTS):
    MATRIX_GLOBAL_DOF = np.zeros((TOTAL_NODES,3))
    for I_COUNT in range (TOTAL_NODES):
        if TYPE_ELEMENT == "BAR_2D_1DOF":  
            MATRIX_GLOBAL_DOF[I_COUNT, 0] = int(TOTAL_DOF_NODE * I_COUNT + 0)
            MATRIX_GLOBAL_DOF[I_COUNT, 1] = -1989
            MATRIX_GLOBAL_DOF[I_COUNT, 2] = -1989
        elif TYPE_ELEMENT == "TRUSS_2D_4DOF":   
            MATRIX_GLOBAL_DOF[I_COUNT, 0] = int(TOTAL_DOF_NODE * I_COUNT + 0)
            MATRIX_GLOBAL_DOF[I_COUNT, 1] = int(TOTAL_DOF_NODE * I_COUNT + 1)
            MATRIX_GLOBAL_DOF[I_COUNT, 2] = -1989
        elif TYPE_ELEMENT == "BEAM_2D_4DOF":   
            MATRIX_GLOBAL_DOF[I_COUNT, 0] = -1989
            MATRIX_GLOBAL_DOF[I_COUNT, 1] = int(TOTAL_DOF_NODE * I_COUNT + 0)
            MATRIX_GLOBAL_DOF[I_COUNT, 2] = int(TOTAL_DOF_NODE * I_COUNT + 1)
        elif TYPE_ELEMENT == "FRAME_2D_6DOF": 
            MATRIX_GLOBAL_DOF[I_COUNT, 0] = int(TOTAL_DOF_NODE * I_COUNT + 0)
            MATRIX_GLOBAL_DOF[I_COUNT, 1] = int(TOTAL_DOF_NODE * I_COUNT + 1)
            MATRIX_GLOBAL_DOF[I_COUNT, 2] = int(TOTAL_DOF_NODE * I_COUNT + 2)
    return MATRIX_GLOBAL_DOF

def TOTAL_DEGREE_FREEDOM(TOTAL_DOF_NODE, TOTAL_NODES):
    ARRAY_TOTAL_DOF = []
    TOTAL_DOF = TOTAL_NODES*TOTAL_DOF_NODE
    for I_COUNT in range (TOTAL_DOF):
        ARRAY_TOTAL_DOF.append (I_COUNT)
    return ARRAY_TOTAL_DOF, TOTAL_DOF

def PRESCRIBED_DEGREE_FREEDOM(MATRIX_NODAL_PRESCRIPTIONS, MATRIX_GLOBAL_DOF, TOTAL_DOF_NODE):
    ARRAY_PRESCRIBED_DOF = []
    ARRAY_PRESCRIBED_DOF_VALUE = []
    TOTAL_PRESCRIBED_DOF = MATRIX_NODAL_PRESCRIPTIONS.shape[0]
    for I_COUNT in range(TOTAL_PRESCRIBED_DOF):
        NODE = MATRIX_NODAL_PRESCRIPTIONS[I_COUNT, 0]
        INDEX_DOF = int(MATRIX_NODAL_PRESCRIPTIONS[I_COUNT, 1])
        DOF_VALUE = int(MATRIX_GLOBAL_DOF[NODE, INDEX_DOF])
        ARRAY_PRESCRIBED_DOF.append(DOF_VALUE)
        PRESCRIBED_VALUE = MATRIX_NODAL_PRESCRIPTIONS[I_COUNT, 2]
        ARRAY_PRESCRIBED_DOF_VALUE.append(PRESCRIBED_VALUE)
    return TOTAL_PRESCRIBED_DOF, ARRAY_PRESCRIBED_DOF_VALUE, ARRAY_PRESCRIBED_DOF

def FREE_DEGREE_FREEDOM(ARRAY_PRESCRIBED_DOF, ARRAY_TOTAL_DOF):
    ARRAY_FREE_DOF = np.setdiff1d(ARRAY_TOTAL_DOF, ARRAY_PRESCRIBED_DOF)
    TOTAL_FREE_DOF = len(ARRAY_FREE_DOF)
    return TOTAL_FREE_DOF, ARRAY_FREE_DOF

def NODAL_EXTERNAL_LOAD(MATRIX_EXTERNAL_FORCE, TOTAL_NODAL_LOADS, TOTAL_DOF, MATRIX_GLOBAL_DOF):
    COMPLETE_FORCE_ARRAY_NODAL_CONTRIBUTION = np.zeros((TOTAL_DOF,1))
    for I_COUNT in range(TOTAL_NODAL_LOADS):
        NODE = int(MATRIX_EXTERNAL_FORCE[I_COUNT, 0])
        INDEX_DOF = int(MATRIX_EXTERNAL_FORCE[I_COUNT, 1])
        DOF_VALUE = int(MATRIX_GLOBAL_DOF[NODE, INDEX_DOF])
        LOAD = int(MATRIX_EXTERNAL_FORCE[I_COUNT, 2])
        COMPLETE_FORCE_ARRAY_NODAL_CONTRIBUTION [DOF_VALUE, 0] = LOAD
    return COMPLETE_FORCE_ARRAY_NODAL_CONTRIBUTION

def MATERIALS_PROPRETIES(MATRIX_ELEMENT_PROPERTIES, MATRIX_MATERIAL_PROPERTIES, I_ELEMENT, INDEX_COLUMN_MAT):
    MATERIAL_INDEX = MATRIX_ELEMENT_PROPERTIES[I_ELEMENT, INDEX_COLUMN_MAT]
    E_MODULUS_VALUE = MATRIX_MATERIAL_PROPERTIES[MAT, 0]
    POISSON_VALUE = MATRIX_MATERIAL_PROPERTIES[MAT, 1]
    THERMAL_COEFFICIENT = MATRIX_MATERIAL_PROPERTIES[MAT, 2]
    G_MODULUS_VALUE = E_MODULUS_VALUE / (2 * (1 + POISSON_VALUE))
    ARRAY_ELEMENT_MATERIAL = [E_MODULUS_VALUE, G_MODULUS_VALUE, POISSON_VALUE, THERMAL_COEFFICIENT]
    return ARRAY_ELEMENT_MATERIAL

def GEOMETRIC_PROPRETIES(MATRIX_COORDINATES_PROPERTIES, MATRIX_ELEMENT_PROPERTIES, SECTION_MATRIX_PROPERTIES, I_ELEMENT, INDEX_COLUMN_SEC):
    NODE_1 = MATRIX_ELEMENT_PROPERTIES(I_ELEMENT, 0)
    NODE_2 = MATRIX_ELEMENT_PROPERTIES(I_ELEMENT, 1)
    X_NODE_1 = MATRIX_COORDINATES_PROPERTIES(NODE_1, 0)
    Y_NODE_1 = MATRIX_COORDINATES_PROPERTIES(NODE_1, 1)
    X_NODE_2 = MATRIX_COORDINATES_PROPERTIES(NODE_2, 0)
    Y_NODE_2 = MATRIX_COORDINATES_PROPERTIES(NODE_3, 1)
    DELTA_X = X_NODE_2 - X_NODE_1
    DELTA_Y = Y_NODE_2 - Y_NODE_1
    LENGTHH = ((DELTA_X) ** 2 + (DELTA_Y) ** 2) ** 0.50
    COSS = DELTA_X / LENGTHH
    SINN = DELTA_Y / LENGTHH
    SEC = MATRIX_ELEMENT_PROPERTIES[I_ELEMENT, INDEX_COLUMN_SEC]
    AREA_VALUE = SECTION_MATRIX_PROPERTIES[SEC, 0]
    INERTIAX_VALUE = SECTION_MATRIX_PROPERTIES[SEC, 1]
    INERTIAY_VALUE = SECTION_MATRIX_PROPERTIES[SEC, 2]
    ARRAY_ELEMENT_GEOMETRY = [LENGTHH, SINN, COSS, AREA_VALUE, INERTIAX_VALUE, INERTIAY_VALUE]
    return ARRAY_ELEMENT_GEOMETRY

def ELEMENTAR_STIFFNESS(TYPE_ELEMENT, ELEMENT_LENGTH, ARRAY_ELEMENT_GEOMETRY, ARRAY_ELEMENT_MATERIAL):
    if TYPE_ELEMENT == "BAR_2D_1DOF":
        L = ARRAY_ELEMENT_GEOMETRY[0]
        A = ARRAY_ELEMENT_GEOMETRY[3]
        E = ARRAY_ELEMENT_MATERIAL[0]
        MATRIX_ELEMENTAR_STIFFNESS = (A * E / L) * np.array([[1, -1],
                                                            [-1, 1]])        
    elif TYPE_ELEMENT == "TRUSS_2D_4DOF":
            L = ARRAY_ELEMENT_GEOMETRY[0]
            A = ARRAY_ELEMENT_GEOMETRY[3]
            E = ARRAY_ELEMENT_MATERIAL[0]
            MATRIX_ELEMENTAR_STIFFNESS = (A * E / L) * np.array([[1, 0,-1, 0],
                                                                [0, 0, 0, 0],
                                                                [-1, 0, 1, 0],
                                                                [0, 0, 0, 0]])
    elif TYPE_ELEMENT == "BEAM_2D_4DOF":
            L = ARRAY_ELEMENT_GEOMETRY[0]
            I = ARRAY_ELEMENT_GEOMETRY[5]
            E = ARRAY_ELEMENT_MATERIAL[0]
            MATRIX_ELEMENTAR_STIFFNESS = (E * I) * np.array([[12 / L ** 3,  6 / L ** 2, -12 / L ** 3,  6 / L ** 2],
                                                            [6 / L ** 2, 4 / L, -6 / L ** 2, 2 / L],
                                                            [-12 / L ** 3, -6 / L ** 2, 12 / L ** 3, -6 / L ** 2],
                                                            [6 / L ** 2, 2 / L, -6 / L ** 2, 4 / L]])
    elif TYPE_ELEMENT == "FRAME_2D_6DOF":
            L = ARRAY_ELEMENT_GEOMETRY[0]
            A = ARRAY_ELEMENT_GEOMETRY[3]
            I = ARRAY_ELEMENT_GEOMETRY[5]
            E = ARRAY_ELEMENT_MATERIAL[0]
            C1 = A * E / L
            C2 = E * I / (L **3)
            MATRIX_ELEMENTAR_STIFFNESS = np.array([[C1, 0, 0, -C1, 0, 0],
                                        [0, 12 * C2, 6 * C2 * L, 0, -12 * C2, 6 * C2 * L],
                                        [0, 6 * C2 * L, 4 * C2 * L ** 2, 0, -6 * C2 * L, 2 * C2 * L ** 2],
                                        [-C1, 0, 0, C1, 0, 0],
                                        [0, -12 * C2, -6 * C2 * L, 0, 12 * C2, -6 * C2 * L],
                                        [0, 6 * C2 * L, 2 * C2 * L ** 2, 0, -6 * C2 * L, 4 * C2 * L **2]])
    return MATRIX_ELEMENTAR_STIFFNESS

def ROTATION(TYPE_ELEMENT, ARRAY_ELEMENT_GEOMETRY):
    SINN = ARRAY_ELEMENT_GEOMETRY[1]
    COSS = ARRAY_ELEMENT_GEOMETRY[2]
    if TYPE_ELEMENT == "BAR_2D_1DOF":
        MATRIX_TRANSFORM = np.array([[1 , 0],
                                    [0, 1]])
    elif TYPE_ELEMENT == "TRUSS_2D_4DOF":
            MATRIX_TRANSFORM = np.array([[ COSS, SINN, 0, 0],
                                        [-SINN, COSS, 0, 0],
                                        [ 0, 0, COSS, SINN],
                                        [0, 0, -SINN, COSS]])
    elif TYPE_ELEMENT == "BEAM_2D_4DOF":
            MATRIX_TRANSFORM = np.array([[1 , 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1]])
    elif TYPE_ELEMENT == "FRAME_2D_6DOF":
            MATRIX_TRANSFORM = np.array([[COSS, SINN, 0, 0, 0, 0],
                                        [-SINN,COSS, 0, 0, 0, 0],
                                        [0, 0, 1, 0, 0, 0],
                                        [0, 0, 0, COSS, SINN, 0],
                                        [0, 0, 0, -SINN, COSS, 0],
                                        [0, 0, 0, 0, 0, 1]])
    return MATRIX_TRANSFORM

def GLOBAL_DOF_ELEMENT(TOTAL_NODES_ELEMENT, TOTAL_DOF_NODE, MATRIX_GLOBAL_DOF, MATRIX_ELEMENT_PROPERTIES, I_ELEMENT):
    if TYPE_ELEMENT == "BAR_2D_1DOF":
        ARRAY_LOCAL_DOF = [0, 1]
    elif TYPE_ELEMENT == "TRUSS_2D_4DOF" and TYPE_ELEMENT == "BEAM_2D_4DOF":
            ARRAY_LOCAL_DOF = [0, 1, 2, 3]
    elif TYPE_ELEMENT == "FRAME_2D_6DOF":
            ARRAY_LOCAL_DOF = [0, 1, 2, 3, 4, 5]
    ARRAY_GLOBAL_DOF = []
    for I_COUNT in range(TOTAL_NODES_ELEMENT):
        NODE = MATRIX_ELEMENT_PROPERTIES[I_ELEMENT, I_COUNT]
        for J_COUNT in range(TOTAL_DOF_NODE):
            DOF_VALUE = int(MATRIX_GLOBAL_DOF[NODE, J_COUNT])
            ARRAY_PRESCRIBED_DOF.append(DOF_VALUE)
    return ARRAY_LOCAL_DOF, ARRAY_GLOBAL_DOF

def GLOBAL_STIFFNESS(ARRAY_GLOBAL_DOF, MATRIX_GLOBAL_STIFFNESS, MATRIX_ELEMENTAR_STIFFNESS):
    for I_COUNT, I_VALUE in enumerate(ARRAY_GLOBAL_DOF):
        for J_COUNT, J_VALUE in enumerate(ARRAY_GLOBAL_DOF):
            MATRIX_GLOBAL_STIFFNESS[I_VALUE, J_VALUE] = MATRIX_GLOBAL_STIFFNESS[I_VALUE, J_VALUE] + MATRIX_ELEMENTAR_STIFFNESS[I_COUNT, J_COUNT]
    return MATRIX_GLOBAL_STIFFNESS

In [2]:
import numpy as np
typeElement         = "TRUSS_2D_4DOF"
numberNodes         = 6
Mcoor               =  np.array([[18.28, 9.14], [18.28, 0.0], [9.14, 9.14], [9.14, 0.0], [0.0, 9.14], [0.0, 0.0]])
Mpele               =  np.array([[5, 3, 1, 1], [3, 1, 1, 1], [6, 4, 1, 1], [4, 2, 1, 1], [4, 3, 1, 1], [2, 1, 1, 1], [5, 4, 1, 1], [6, 3, 1, 1], [4, 1, 1, 1], [3, 2, 1, 1]]) 
numberElements      = 10
prescribedMatrix    = np.array([[4, 0, 0],[4, 1, 0],[5, 0, 0],[5, 1, 0]])
externalForces      = np.array([[1, 1, -450e3],[3, 1, -450e3]])
totalNodalForces 	= 2

print("     ")
print("linha")
[totalDOFNode, totalNodesElement, totalDOFElement,  columnMaterial, columnSecao, arrayDOFActive] = INDEX_ASSEMBLY(typeElement)

print("totalDOFNode", totalDOFNode)
print("totalNodesElement", totalNodesElement)
print("totalDOFElement", totalDOFElement)
print("columnMaterial", columnMaterial)
print("columnSecao", columnSecao)
print("arrayDOFActive", arrayDOFActive)

print("     ")
print("linha")
nodesGlobalDOF = GLOBAL_DOF_ASSEMBLY(typeElement, totalDOFNode, numberNodes, numberElements)
print("nodesGlobalDOF")
print(nodesGlobalDOF)

print("     ")
print("linha")
[totalDOFArray, totalDOF] = TOTAL_DEGREE_FREEDOM(totalDOFNode, numberNodes)
print("totalDOF", totalDOF)
print("totalDOFArray")
print(totalDOFArray)

print("     ")
print("linha")
[numberDOFPrescribed, valuePrescribedDOFArray, totalPrescribedDOFArray] = PRESCRIBED_DEGREE_FREEDOM(prescribedMatrix, nodesGlobalDOF, totalDOFNode)
print("numberDOFPrescribed", numberDOFPrescribed)
print("valuePrescribedDOFArray")
print(valuePrescribedDOFArray)
print("totalPrescribedDOFArray")
print(totalPrescribedDOFArray)

print("     ")
print("linha")
[numberDOFFree, totalFreeDOFArray] = FREE_DEGREE_FREEDOM(totalPrescribedDOFArray, totalDOFArray)
print("numberDOFFree", numberDOFFree)
print("totalFreeDOFArray")
print(totalFreeDOFArray)

print("     ")
print("linha")
nodalForceContribuition = NODAL_EXTERNAL_LOAD(externalForces, totalNodalForces, totalDOF, nodesGlobalDOF)
print("nodalForceContribuition")
print(nodalForceContribuition)

     
linha
totalDOFNode 2
totalNodesElement 2
totalDOFElement 4
columnMaterial 2
columnSecao 3
arrayDOFActive [[1 1 0]]
     
linha
nodesGlobalDOF
[[ 0.000e+00  1.000e+00 -1.989e+03]
 [ 2.000e+00  3.000e+00 -1.989e+03]
 [ 4.000e+00  5.000e+00 -1.989e+03]
 [ 6.000e+00  7.000e+00 -1.989e+03]
 [ 8.000e+00  9.000e+00 -1.989e+03]
 [ 1.000e+01  1.100e+01 -1.989e+03]]
     
linha
totalDOF 12
totalDOFArray
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
     
linha
numberDOFPrescribed 4
valuePrescribedDOFArray
[0, 0, 0, 0]
totalPrescribedDOFArray
[8, 9, 10, 11]
     
linha
numberDOFFree 8
totalFreeDOFArray
[0 1 2 3 4 5 6 7]
     
linha
nodalForceContribuition
[[      0.]
 [      0.]
 [      0.]
 [-450000.]
 [      0.]
 [      0.]
 [      0.]
 [-450000.]
 [      0.]
 [      0.]
 [      0.]
 [      0.]]
